# Homework №1

### <center>Group project name: *{Put name here}*</center>

### NB! It is important to send only one work from group (done by group leader)

# Description

1. Those how are making projects on artilces need to analyze both co-authorship and citation graphs
2. Provide analysis for graphs supposed them to be directed and weighted (where it is applicable)
3. Make all your analysis and visualization work as buit-into your web-service
4. It's preferable to use Gephi, GraphViz or D3.JS on visualization purpose

### Guidelines:

1. Make graph drawing (applying possibility of filtering as it was described in projects) using at least **3 different types of layouts** (different modifications of spring layouts are to be counted as one)

2. Print **key graph metrics** (such as density, diameter etc.)

3. Output some **interesting facts** from network (e.g. the most central author, TOP-3 citated articles etc.)

4. Suggest some kind of **community detection** (perhaps, based on idea of eigenvalues and eigenvectors) and visualize it (on demand)

